In [1]:
# img_labels & img_list
PATH_TO_LABELS_FILE = "../data/output_dataset/img_labels"
PATH_TO_PATHS_FILE = "../data/output_dataset/img_list"

# output convnet
PATH_TO_OUTPUT = "../data/output_convnet"

img_labels = []
img_list = []

with open(PATH_TO_LABELS_FILE,'r') as f_img_labels:
    for line in f_img_labels:
        img_labels.append(line[:-1]) 

with open(PATH_TO_PATHS_FILE,'r') as f_img_list:
    for line in f_img_list:
        img_list.append("../data/"+line[:-1]) # get rid of the EOL

# fc1 layer with activation relU

In [2]:
from keras.models import Model
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, VGG16
import numpy as np

Using TensorFlow backend.


In [3]:
# keras_weights = "keras_weights/vgg16_weights_tf_dim_ordering_tf_kernels.h5"
model_relu = VGG16(include_top=True,weights='imagenet')

# model architecture
for i, layer in enumerate(model_relu.layers):
    print (i, layer.name, layer.output_shape)

0 input_1 (None, 224, 224, 3)
1 block1_conv1 (None, 224, 224, 64)
2 block1_conv2 (None, 224, 224, 64)
3 block1_pool (None, 112, 112, 64)
4 block2_conv1 (None, 112, 112, 128)
5 block2_conv2 (None, 112, 112, 128)
6 block2_pool (None, 56, 56, 128)
7 block3_conv1 (None, 56, 56, 256)
8 block3_conv2 (None, 56, 56, 256)
9 block3_conv3 (None, 56, 56, 256)
10 block3_pool (None, 28, 28, 256)
11 block4_conv1 (None, 28, 28, 512)
12 block4_conv2 (None, 28, 28, 512)
13 block4_conv3 (None, 28, 28, 512)
14 block4_pool (None, 14, 14, 512)
15 block5_conv1 (None, 14, 14, 512)
16 block5_conv2 (None, 14, 14, 512)
17 block5_conv3 (None, 14, 14, 512)
18 block5_pool (None, 7, 7, 512)
19 flatten (None, 25088)
20 fc1 (None, 4096)
21 fc2 (None, 4096)
22 predictions (None, 1000)


In [4]:
import time
from sys import getsizeof

deep_features = 4096
VGG16_dfmap = np.empty((len(img_list),deep_features))

# init time
ti_dfmap_relU = time.time()

for index, img_path in enumerate(img_list):
        
    model_fc1 = Model(input=model_relu.input, output=model_relu.get_layer('fc1').output)

    # img preprocessing
    img = image.load_img(img_path, target_size=(224, 224), interpolation='lanczos')
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    # model predict
    x = model_fc1.predict(x)
    
    VGG16_dfmap[index,:] = np.squeeze(x, axis=0)

# stop time
tf_dfmap_relU = time.time()    
tt_dfmap_relU = tf_dfmap_relU - ti_dfmap_relU
print()
print(time.strftime("%H:%M:%S", time.gmtime(tt_dfmap_relU)))
print()
print("Tamaño en memoria de la matriz de características profundas: {0:.2f}Mb".format(getsizeof(VGG16_dfmap)/float(1<<20)))

image 0 processed


/home/jon/anaconda3/envs/test/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("fc...)`
  from ipykernel import kernelapp as app


image 500 processed
image 1000 processed
image 1500 processed
image 2000 processed
image 2500 processed
image 3000 processed
image 3500 processed
image 4000 processed
image 4500 processed

00:08:41

Tamaño en memoria de la matriz de características profundas: 156.25Mb


In [10]:
# checking the output with relu funct
VGG16_dfmap

array([[ 1.4363389 ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.47403866,
         0.        ,  0.        ],
       [ 7.5820446 ,  0.        ,  0.        , ...,  9.63796329,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  1.40950263,  0.        , ...,  0.21009497,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.99256498,
         0.        ,  0.        ],
       [ 0.        ,  0.57167876,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [6]:
# import h5py
import os
import pickle


# h5py alternative
# ti_h5file = time.time()

# hf = h5py.File(os.path.join(PATH_TO_OUTPUT, 'VGG16_dfmap_relU.h5'), 'w')
# hf.create_dataset('dfmap', data=VGG16_dfmap) #163.8 mb no compression at all
# # hf.create_dataset('dfmap', data=VGG16_dfmap,compression="gzip", compression_opts=9) # 32.4 mb
# hf.close()

# # stop time
# tf_h5file = time.time()
# tt_h5file = tf_h5file - ti_h5file
# print(time.strftime("%H:%M:%S", time.gmtime(tt_h5file)))

pickle.dump(VGG16_dfmap, open('../data/output_convnet/VGG16/VGG16_dfmap_relU_pickle', 'wb'))
print(os.path.getsize("../data/output_convnet/VGG16/VGG16_dfmap_relU_pickle")/float(1<<20)) #156 mb

156.25015449523926


In [ ]:
# file information h5py alternative
# import h5py

# hf = h5py.File(os.path.join(PATH_TO_OUTPUT, 'VGG16_dfmap_relU.h5'), 'r')
# print(hf.keys())
# n1 = hf.get('dfmap')
# print(type(n1))
# print(n1)

# n1 = np.array(n1)
# print(type(n1))
# print(n1.shape)

# hf.close()

# fc1 layer with no activation function

In [7]:
from keras import backend as K
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from keras.models import Sequential

In [8]:
# https://github.com/keras-team/keras/blob/master/keras/applications/vgg16.py

model_no_relu = Sequential()
model_no_relu.add(ZeroPadding2D((1,1),input_shape=(224,224,3)))
model_no_relu.add(Conv2D(64, (3, 3), activation='relu'))
model_no_relu.add(ZeroPadding2D((1,1)))
model_no_relu.add(Conv2D(64, (3, 3), activation='relu'))
model_no_relu.add(MaxPooling2D((2,2), strides=(2,2)))

model_no_relu.add(ZeroPadding2D((1,1)))
model_no_relu.add(Conv2D(128, (3, 3), activation='relu'))
model_no_relu.add(ZeroPadding2D((1,1)))
model_no_relu.add(Conv2D(128, (3, 3), activation='relu'))
model_no_relu.add(MaxPooling2D((2,2), strides=(2,2)))

model_no_relu.add(ZeroPadding2D((1,1)))
model_no_relu.add(Conv2D(256, (3, 3), activation='relu'))
model_no_relu.add(ZeroPadding2D((1,1)))
model_no_relu.add(Conv2D(256, (3, 3), activation='relu'))
model_no_relu.add(ZeroPadding2D((1,1)))
model_no_relu.add(Conv2D(256, (3, 3), activation='relu'))
model_no_relu.add(MaxPooling2D((2,2), strides=(2,2)))

model_no_relu.add(ZeroPadding2D((1,1)))
model_no_relu.add(Conv2D(512, (3, 3), activation='relu'))
model_no_relu.add(ZeroPadding2D((1,1)))
model_no_relu.add(Conv2D(512, (3, 3), activation='relu'))
model_no_relu.add(ZeroPadding2D((1,1)))
model_no_relu.add(Conv2D(512, (3, 3), activation='relu'))
model_no_relu.add(MaxPooling2D((2,2), strides=(2,2)))

model_no_relu.add(ZeroPadding2D((1,1)))
model_no_relu.add(Conv2D(512, (3, 3), activation='relu'))
model_no_relu.add(ZeroPadding2D((1,1)))
model_no_relu.add(Conv2D(512, (3, 3), activation='relu'))
model_no_relu.add(ZeroPadding2D((1,1)))
model_no_relu.add(Conv2D(512, (3, 3), activation='relu'))
model_no_relu.add(MaxPooling2D((2,2), strides=(2,2)))

model_no_relu.add(Flatten())

# top layer of the VGG net
model_no_relu.add(Dense(4096, activation=None,name="fc1")) # activation function set to none
model_no_relu.add(Dropout(0.5))
model_no_relu.add(Dense(4096, activation=None,name="fc2")) # activation function set to none
model_no_relu.add(Dropout(0.5))
model_no_relu.add(Dense(1000, activation='softmax'))

model_no_relu.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_1 (ZeroPaddin (None, 226, 226, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 226, 226, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 114, 114, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     73856     
__________

In [9]:
model_no_relu.load_weights("keras_weights/vgg16_weights_tf_dim_ordering_tf_kernels.h5")
model_no_relu.compile(optimizer=SGD(lr=0.01), loss='categorical_crossentropy')

In [11]:
VGG16_dfmap_no_relU = np.empty((len(img_list),deep_features))

# init time
ti_dfmap_no_relU = time.time()

for index, img_path in enumerate(img_list):

    model_fc1 = Model(input=model_no_relu.input, output=model_no_relu.get_layer('fc1').output) # no relu implemented

    # img preprocessing
    img = image.load_img(img_path, target_size=(224, 224), interpolation='lanczos')
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    # model predict
    x = model_fc1.predict(x)
    
    VGG16_dfmap_no_relU[index,:] = np.squeeze(x, axis=0)

# stop time
tf_dfmap_no_relU = time.time()    
tt_dfmap_no_relU = tf_dfmap_no_relU - ti_dfmap_no_relU
print()
print(time.strftime("%H:%M:%S", time.gmtime(tt_dfmap_no_relU)))
print()
print("Tamaño en memoria de la matriz de características profundas: {0:.2f}Mb".format(getsizeof(VGG16_dfmap_no_relU)/float(1<<20)))

/home/jon/anaconda3/envs/test/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("ze..., outputs=Tensor("fc...)`
  



00:15:25

Tamaño en memoria de la matriz de características profundas: 156.25Mb


In [12]:
# # checking the output with no activation funct
VGG16_dfmap_no_relU 

array([[  1.4363389 ,  -9.70764256, -14.33625031, ...,  -0.2988835 ,
         -4.04614496,  -2.43703127],
       [ -6.25107241,  -8.4447031 , -11.96656322, ...,   0.47403866,
         -4.45992088,  -8.00944138],
       [  7.5820446 , -21.92349815,  -4.70616055, ...,   9.63796329,
         -3.06682301, -17.91776276],
       ..., 
       [ -4.1283865 ,   1.40950263,  -4.15737724, ...,   0.21009497,
         -0.19954918,  -1.71088219],
       [ -3.73391938,  -0.72124833,  -5.90556002, ...,   0.99256498,
         -0.52596283,  -2.18934989],
       [ -2.97715545,   0.57167876,  -6.36512375, ...,  -1.06647027,
         -1.3277365 ,  -3.27218819]])

In [14]:
# import pickle

# # init time
# ti_h5file_no_relU = time.time()

# hf = h5py.File(os.path.join(PATH_TO_OUTPUT, 'VGG16_dfmap_no_relU.h5'), 'w')
# hf.create_dataset('dfmap', data=VGG16_dfmap) #163.8 mb
# # hf.create_dataset('dfmap', data=VGG16_dfmap_no_relU,compression="gzip", compression_opts=9) # 32.4 mb
# hf.close()

# # stop time
# tf_h5file_no_relU = time.time()
# tt_h5file_no_relU = tf_h5file_no_relU - ti_h5file_no_relU
# print(time.strftime("%H:%M:%S", time.gmtime(tt_h5file_no_relU)))


pickle.dump(VGG16_dfmap_no_relU, open('../data/output_convnet/VGG16/VGG16_dfmap_no_relU_pickle', 'wb'))
print(os.path.getsize("../data/output_convnet/VGG16/VGG16_dfmap_no_relU_pickle")/float(1<<20)) #156 mb

156.25015449523926


In [ ]:
# hf = h5py.File(os.path.join(PATH_TO_OUTPUT, 'VGG16_dfmap_no_relU.h5'), 'r')
# print(hf.keys())
# n1 = hf.get('dfmap')
# print(type(n1))
# print(n1)

# n1 = np.array(n1)
# print(type(n1))
# print(n1.shape)

# hf.close()